In [1]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

from datetime import *
today = date.today()
today = today.strftime("%m.%d.20%y")
today

# import datetime
# today = date.today()
# today = today - datetime.timedelta(days=1)
# today = today.strftime("%m.%d.20%y")
# today

'03.16.2022'

In [6]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

fd = glob.glob('FanDuel-NBA*.csv')[0]
os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [3]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/vchang/Downloads/FD NBA Projections.csv'

In [2]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [3]:
# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)

# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list
0,Giannis Antetokounmpo,MIL,SAC,34.0,PF/SF,11400.0,57.7,7.1,67.4,29.4,MAIN,Giannis Antetokounmpo,PF/SF,73027-40199,NaN,F,PF/SF/F
1,Domantas Sabonis,SAC,MIL,34.0,PF/C,8400.0,45.1,6.5,53.8,22.0,MAIN,Domantas Sabonis,PF/C,73027-59358,NaN,F,PF/C/F
2,Donovan Mitchell,UTA,CHI,37.0,SG/PG,8300.0,44.5,6.3,53.9,27.8,MAIN,Donovan Mitchell,SG/PG,73027-66113,G,NaN,SG/PG/G
3,Rudy Gobert,UTA,CHI,32.0,C,7300.0,39.2,5.0,47.1,8.2,MAIN,Rudy Gobert,C,73027-40203,NaN,NaN,C
4,Jaylen Brown,BOS,GSW,35.0,SF/SG,6800.0,37.0,4.8,46.7,25.8,MAIN,Jaylen Brown,SF/SG,73027-67026,G,F,SF/SG/G/F


In [4]:
df.to_excel("FD_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)

df = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_PlayerPool_ETR_'+ str(today) + ".xlsx")

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list
0,Giannis Antetokounmpo,MIL,SAC,34.0,PF/SF,11400,57.7,7.1,67.4,29.4,MAIN,Giannis Antetokounmpo,PF/SF,73027-40199,NaN,F,PF/SF/F
1,Domantas Sabonis,SAC,MIL,34.0,PF/C,8400,45.1,6.5,53.8,22.0,MAIN,Domantas Sabonis,PF/C,73027-59358,NaN,F,PF/C/F
2,Donovan Mitchell,UTA,CHI,37.0,SG/PG,8300,44.5,6.3,53.9,27.8,MAIN,Donovan Mitchell,SG/PG,73027-66113,G,NaN,SG/PG/G
3,Rudy Gobert,UTA,CHI,32.0,C,7300,39.2,5.0,47.1,8.2,MAIN,Rudy Gobert,C,73027-40203,NaN,NaN,C
4,Jaylen Brown,BOS,GSW,35.0,SF/SG,6800,37.0,4.8,46.7,25.8,MAIN,Jaylen Brown,SF/SG,73027-67026,G,F,SF/SG/G/F


# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_C


In [5]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [6]:
# number of players constraint
prob += pulp.lpSum(player_var for player_var in player_vars) == 9

# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['list'].iloc[i]) for i in range(len(df))])
    
# prob += get_position_sum(player_vars, df, 'PG') <= 1
prob += get_position_sum(player_vars, df, 'PG') >= 2
# prob += get_position_sum(player_vars, df, 'SG') <= 3
prob += get_position_sum(player_vars, df, 'SG') >= 2
prob += get_position_sum(player_vars, df, 'G') >= 4
# prob += get_position_sum(player_vars, df, 'SF') <= 2
prob += get_position_sum(player_vars, df, 'SF') >= 2
# prob += get_position_sum(player_vars, df, 'PF') <= 2
prob += get_position_sum(player_vars, df, 'PF') >= 2
prob += get_position_sum(player_vars, df, 'F') >= 4
# prob += get_position_sum(player_vars, df, 'C') <= 1
prob += get_position_sum(player_vars, df, 'C') >= 1


# total salary constraint
prob += pulp.lpSum(df['FD Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 60000

# finally, specify the goal
prob += pulp.lpSum([df['FD Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [7]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [8]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['list'], row.Player, row.Team, row['FD Salary'], row['FD Points'])
        total_salary_used += row['FD Salary']
        mean_AvgPointsPerGame += row['FD Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

PF/SF/F Giannis Antetokounmpo MIL 11400 57.7
PF/C/F Domantas Sabonis SAC 8400 45.1
SG/PG/G Donovan Mitchell UTA 8300 44.5
C Rudy Gobert UTA 7300 39.2
PG/SG/G Jordan Clarkson UTA 5000 29.7
PG/G Fred VanVleet TOR 7500 39.6
SF/SG/G/F Amir Coffey LAC 4000 21.6
SF/SG/G/F Justin Holiday SAC 3500 19.5
SF/PF/F Royce O'Neale UTA 4600 23.0


(60000, 319.9)